## 0. 필요 라이브러리 

In [20]:
import bs4
from bs4 import BeautifulSoup 
import requests
import urllib
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
import json
import re
import warnings
warnings.filterwarnings(action='ignore')

## 1. 교보문고 크롤링
- 페이지 검사 > 네트워크에서 해당 페이지의 순위 정보에 대한 json에 바로 접근할 수 있는 링크가 존재함
- 따라서, selenium으로 클릭하지 않고, for문으로 1부터 20까지 페이지 넘버를 넣어 json 파일 받을 수 있는 방식 채택
- 소제목에 해당하는 정보가 없어 소제목 항목은 제외하였습니다.
- 이벤트 정보의 경우에도, 정확하게 해당 이름으로 존재하는 항목이 없어, 세일 가격을 기입했습니다.
- 추가정보의 경우, 해당 책에 대한 소개글을 기입했습니다

In [23]:
rankings = pd.DataFrame(columns =['순위','제목','상세보기 링크',
'이벤트 정보','저자','출판사','출판일자','가격','추가정보'] )

In [26]:
url = 'https://product.kyobobook.co.kr/api/gw/pub/pdt/best-seller/total?page=%s&per=20&period=002&bsslBksClstCode=A'

for i in tqdm(range(1,11, 1)):
    source = requests.get(url%str(i))
    #print(source.text)
    json_ob = json.loads(source.text)
    json_ob = json_ob['data']
    best = json_ob['bestSeller']
    
    for k in range(len(best)):
        row = {'순위':best[k]['prstRnkn'],
                '제목' :best[k]['cmdtName'],
                '상세보기 링크': "https://product.kyobobook.co.kr/detail/"+best[k]['saleCmdtid'],
                '이벤트 정보' : best[k]['sapr'],
                '저자' : best[k]['chrcName'],
                '출판사' : best[k]['pbcmName'],
                '출판일자' : best[k]['rlseDate'],
                '가격' : best[k]['price'],
                '추가정보' : best[k]['inbukCntt']}
        rankings = rankings.append(row, ignore_index= True)


100%|██████████| 10/10 [00:35<00:00,  3.56s/it]


In [27]:
rankings

,순위,제목,상세보기 링크,이벤트 정보,저자,출판사,출판일자,가격,추가정보
0,1,트렌드 코리아 2023,https://product.kyobobook.co.kr/detail/S000061...,17100.0,김난도 외,미래의창,20221005,19000.0,"세계화의 종말, 갈등과 분열, 그리고 전쟁. 수십 년간 이어져 온 평화와 공존의 시..."
1,2,하얼빈,https://product.kyobobook.co.kr/detail/S000061...,14400.0,김훈,문학동네,20220803,16000.0,‘우리 시대 최고의 문장가’ ‘작가들의 작가’로 일컬어지는 소설가 김훈의 신작 장편...
2,3,불편한 편의점(40만부 기념 벚꽃 에디션),https://product.kyobobook.co.kr/detail/S000001...,12600.0,김호연,나무옆의자,20210420,14000.0,"누적 판매 40만부 돌파, 2022년 가장 사랑받는 소설\n★★★전 서점 종합베스트..."
3,4,아버지의 해방일지,https://product.kyobobook.co.kr/detail/S000061...,13500.0,정지아,창비,20220902,15000.0,김유정문학상 심훈문학대상 이효석문학상 등을 수상하며 문학성을 두루 입증받은 ‘리얼리...
4,5,불편한 편의점 2,https://product.kyobobook.co.kr/detail/S000061...,12600.0,김호연,나무옆의자,20220810,14000.0,"출간 후 1년이 넘도록 독자의 사랑을 받으며 베스트셀러 상위권을 지키고 있는 소설,..."
...,...,...,...,...,...,...,...,...,...
195,195,도파민네이션,https://product.kyobobook.co.kr/detail/S000001...,16200.0,애나 렘키 외,흐름출판,20220321,18000.0,도파민네이션(dopamine nation)이란? 과학자들은 중독 가능성을 측정하는 ...
196,195,신경 청소 혁명,https://product.kyobobook.co.kr/detail/S000001...,12600.0,구도 치아키 외,비타북스,20170705,14000.0,39만 명 이상의 환자를 진료한 명의 구도 치아키의 획기적인 건강법! 《신경 청소 ...
197,198,비밀의 보석 가게 마석관 3,https://product.kyobobook.co.kr/detail/S000061...,11700.0,히로시마 레이코 외,길벗스쿨,20221004,13000.0,어린이 판타지 문학의 대표 작가 히로시마 레이코의 「비밀의 보석 가게 마석관」이 3...
198,199,명탐정 코난 101,https://product.kyobobook.co.kr/detail/S000061...,4950.0,Aoyama Gosho,서울미디어코믹스(서울문화사),20220825,5500.0,초등학교에 묻힌 타임캡슐이 있는 장소의 힌트는…\n미야노 아케미가 남긴 암호?! 코...


## 2. 공공데이터포털 표준데이터셋 메타데이터 수집
분류, 데이터명, 설명, 수정일, 조회수, 데이터의 링크

In [ ]:

url2 = 'https://www.data.go.kr/tcs/dss/selectDataSetList.do?dType=STD&keyword=&detailKeyword=&publicDataPk=&recmSe=&detailText=&relatedKeyword=&commaNotInData=&commaAndData=&commaOrData=&must_not=&tabId=&dataSetCoreTf=&coreDataNm=&sort=updtDt&relRadio=&orgFullName=&orgFilter=&org=&orgSearch=&currentPage=%s&perPage=10'
for i in range(1,16,1):
    page = requests.get(url2%str(i))


### 2-1. 세부 페이지 링크 리스트화 + 메타데이터 크롤링
- 목록 페이지에서 개별 세부 페이지의 href를 모두 크롤링하여 리스트화
- 개별 세부 페이지를 request하여 크롤링
- 크롤링할 데이터 : 데이터링크, 조회수, 수정일 

In [146]:
url2 = "https://www.data.go.kr/tcs/dss/selectDataSetList.do?dType=STD&keyword=&detailKeyword=&publicDataPk=&recmSe=&detailText=&relatedKeyword=&commaNotInData=&commaAndData=&commaOrData=&must_not=&tabId=&dataSetCoreTf=&coreDataNm=&sort=updtDt&relRadio=&orgFullName=&orgFilter=&org=&orgSearch=&currentPage=%s&perPage=10&brm=&instt=&svcType=&kwrdArray=&extsn=&coreDataNmArray=&pblonsipScopeCode="
links= pd.DataFrame(columns = ['데이터링크','조회수','수정일'])
acc_links = []
views = []
for i in tqdm(range(1,16,1)):
    html = requests.get(url2%str(i)).text
    soup = BeautifulSoup(html, 'html.parser')
    li = soup.find('div','result-list').find_all('li')
    for k in range(len(li)):
        acc = li[k].a['href']
        acc = 'http://data.go.kr' + acc
        acc_links.append(acc)
        
        view = li[k].find_all('span','data')
        view = view[1]
        view = re.sub('(<([^>]+)>)','', str(view))
        view = view.strip()

        date = li[k].find_all('span','data')
        date = date[0]
        date = re.sub('(<([^>]+)>)','', str(date))
        date = date.strip()

        
        links = links.append({'데이터링크':acc, '조회수':view, '수정일':date}, ignore_index = True)

100%|██████████| 15/15 [00:17<00:00,  1.14s/it]


In [147]:
links

,데이터링크,조회수,수정일
0,http://data.go.kr/data/15012891/standard.do,55094,2022-10-21
1,http://data.go.kr/data/15017320/standard.do,25057,2022-10-21
2,http://data.go.kr/data/15021138/standard.do,21690,2022-10-21
3,http://data.go.kr/data/15021137/standard.do,15049,2022-10-21
4,http://data.go.kr/data/15013117/standard.do,22985,2022-10-21
...,...,...,...
142,http://data.go.kr/data/15013105/standard.do,12107,2022-05-31
143,http://data.go.kr/data/15012893/standard.do,9766,2022-05-31
144,http://data.go.kr/data/15012892/standard.do,33696,2022-05-31
145,http://data.go.kr/data/15013108/standard.do,11255,2022-05-19


### 2-2. 세부 링크 접속하여 크롤링
- 세부 페이지에 하나씩 접근하여 테이블 크롤링
- 데이터명, 설명, 분류체계 크롤링
- 수정일 같은 경우에는 테이블에 항목이 없는 경우도 있어서, 상위 목록에서 크롤링함.

In [140]:
datas = pd.DataFrame(columns = ['분류체계'])
conts = pd.DataFrame(columns = ['데이터명', '설명'])
n = 0
for i in tqdm(range(len(acc_links))):
    # 데이터명, 설명 크롤링
    url = acc_links[i]
    
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')

    
    title = soup.find('p', "tit standard-data-title")
    title = re.sub('(<([^>]+)>)', '', str(title))

    cont = soup.find('div', 'cont')
    cont = re.sub('(<([^>]+)>)', '', str(cont))
    cont = cont.strip()
    conts = conts.append({'데이터명':title, '설명':cont}, ignore_index=True)
    
    table = soup.find_all('td','td custom-cell-border-bottom' )
    table2 = soup.find_all('th', 'th custom-cell-border-bottom custom-cell-border-right')

    # 분류 체계 크롤링
    row = {'분류체계':0}
    for k in range(len(table)):
        td = str(table[k]) # 값
        th = str(table2[k]) # 테이블 항목명

        td = re.sub('(<([^>]+)>)', '', td)
        th = re.sub('(<([^>]+)>)', '', th) 
        
        if th == '분류체계': #테이블 헤드가 '분류체계'인 것만 딕셔너리에 넣음
            row['분류체계'] = td
        
        
    datas = datas.append(row, ignore_index=True)

        


        #print(row)

100%|██████████| 147/147 [04:09<00:00,  1.70s/it]


In [141]:
datas #분류체계

,분류체계
0,공공질서및안전 - 안전관리
1,공공질서및안전 - 안전관리
2,보건 - 보건의료
3,보건 - 보건의료
4,일반공공행정 - 일반행정
...,...
142,문화체육관광 - 관광
143,일반공공행정 - 일반행정
144,환경 - 환경일반
145,사회복지 - 보육·가족및여성


In [142]:
conts #데이터명 및 설명

,데이터명,설명
0,전국어린이보호구역표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
1,전국보안등정보표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
2,전국치매센터표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
3,전국건강증진센터표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
4,전국공공시설개방정보표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
...,...,...
142,전국민박펜션업소표준데이터,"숙박시설을 운영하며, 자연,문화 체험관광에 적합한 시설을 갖추어 제공하는 업소정보 ..."
143,전국무인민원발급정보표준데이터,"""민원처리에 관한 법률"" 등에 따라 행정기관 또는 공공장소 등에 설치하여 민원인이 ..."
144,전국공중화장실표준데이터,"- 공공데이터 제공 표준 기준, 지자체에서 관리하는 공중화장실정보 화장실명, 주소,..."
145,전국어린이집표준데이터,정부가 보유하고 있는 어린이집 주요 정보를 투명하게 공개함으로써 부모들이 보다 쉽고...


### 2-3. 데이터 합치기
분류, 데이터명, 설명, 수정일, 조회수, 데이터의 링크

In [148]:
public_std_meta = pd.concat([conts,datas,links], axis=1)

In [149]:
public_std_meta

,데이터명,설명,분류체계,데이터링크,조회수,수정일
0,전국어린이보호구역표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",공공질서및안전 - 안전관리,http://data.go.kr/data/15012891/standard.do,55094,2022-10-21
1,전국보안등정보표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",공공질서및안전 - 안전관리,http://data.go.kr/data/15017320/standard.do,25057,2022-10-21
2,전국치매센터표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",보건 - 보건의료,http://data.go.kr/data/15021138/standard.do,21690,2022-10-21
3,전국건강증진센터표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",보건 - 보건의료,http://data.go.kr/data/15021137/standard.do,15049,2022-10-21
4,전국공공시설개방정보표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",일반공공행정 - 일반행정,http://data.go.kr/data/15013117/standard.do,22985,2022-10-21
...,...,...,...,...,...,...
142,전국민박펜션업소표준데이터,"숙박시설을 운영하며, 자연,문화 체험관광에 적합한 시설을 갖추어 제공하는 업소정보 ...",문화체육관광 - 관광,http://data.go.kr/data/15013105/standard.do,12107,2022-05-31
143,전국무인민원발급정보표준데이터,"""민원처리에 관한 법률"" 등에 따라 행정기관 또는 공공장소 등에 설치하여 민원인이 ...",일반공공행정 - 일반행정,http://data.go.kr/data/15012893/standard.do,9766,2022-05-31
144,전국공중화장실표준데이터,"- 공공데이터 제공 표준 기준, 지자체에서 관리하는 공중화장실정보 화장실명, 주소,...",환경 - 환경일반,http://data.go.kr/data/15012892/standard.do,33696,2022-05-31
145,전국어린이집표준데이터,정부가 보유하고 있는 어린이집 주요 정보를 투명하게 공개함으로써 부모들이 보다 쉽고...,사회복지 - 보육·가족및여성,http://data.go.kr/data/15013108/standard.do,11255,2022-05-19
